In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [256]:
initial_board_state = [
['3x', '', '', '', '', '', '3x', '3x', '', '', '', '', '', '3x'],
['', '2x', '', '', ':', '', '', '', '', ':', '', '', '2x', ''],
['', '', '2x', '', '', '-', '', '', '-', '', '', '2x', '', ''],
['', '', '', '2x', '', '', '+', '*', '', '', '2x', '', '', ''],
['', ':', '', '', '2x', '', '*', '+', '', '2x', '', '', ':', ''],
['', '', '-', '', '', '', '', '', '', '', '', '-', '', ''],
['3x', '', '', '*', '+', '', 1, 2, '', '*', '+', '', '', '3x'],
['3x', '', '', '+', '*', '', 3, 4, '', '+', '*', '', '', '3x'],
['', '', '-', '', '', '', '', '', '', '', '', '-', '', ''],
['', ':', '', '', '2x', '', '+', '*', '', '2x', '', '', ':', ''],
['', '', '', '2x', '', '', '*', '+', '', '', '2x', '', '', ''],
['', '', '2x', '', '', '-', '', '', '-', '', '', '2x', '', ''],
['', '2x', '', '', ':', '', '', '', '', ':', '', '', '2x', ''],
['3x', '', '', '', '', '', '3x', '3x', '', '', '', '', '', '3x']]
    
column_letter_mapping = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'E',
    5: 'F',
    6: 'G',
    7: 'H',
    8: 'I',
    9: 'J',
    10: 'K',
    11: 'L',
    12: 'M',
    13: 'N'
}

In [101]:
def plot_image(image):
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()

In [75]:
def plot_images(fig, images):
    n_images = len(images)
    cols = 4
    rows = (n_images + cols - 1) // cols 

    for i, (image, title) in enumerate(images, 1):
        ax = fig.add_subplot(rows, cols, i)
        ax.imshow(image)
        ax.set_title(title)
        ax.axis('off')  

    plt.tight_layout()
    plt.show()

In [ ]:
def load_templates(path):
    return None

In [305]:
def get_content_of_piece(board, i, j):
    return None
    

In [307]:
def get_position_of_newly_added_piece(prev_board, curr_board):
    grid_size = 14  # 14x14 board
    tile_size = 100  # each tile is 100x100 pixels
    
    prev_board_gray = cv.cvtColor(prev_board, cv.COLOR_BGR2GRAY)
    curr_board_gray = cv.cvtColor(curr_board, cv.COLOR_BGR2GRAY)
    
    max_diff = 0
    # Uncomment to see the top 5 tiles detected by the algorithm with the highest difference between the previous and current board
    # images = []
    for i in range(grid_size):
        for j in range(grid_size):
            tile_x_start = i * tile_size
            tile_y_start = j * tile_size
            tile_x_end = (i + 1) * tile_size
            tile_y_end = (j + 1) * tile_size
            
            tile_from_prev_board = prev_board_gray[tile_y_start:tile_y_end, tile_x_start:tile_x_end]
            tile_from_curr_board = curr_board_gray[tile_y_start:tile_y_end, tile_x_start:tile_x_end]

            diff = cv.absdiff(tile_from_prev_board, tile_from_curr_board)
            diff_erosion = cv.erode(diff, np.ones((3,3), np.uint8), iterations=1)
            # images.append(diff_erosion)
            diff_mean = np.mean(diff_erosion)

            if diff_mean > max_diff:
                max_diff = diff_mean
                new_piece_position = (j,i)
                
    # images.sort(key=lambda x: np.mean(x), reverse=True)
    # for img in images[:5]:
    #     plot_image(img)
    return new_piece_position

In [303]:
def position_and_content(position, content):
    i, j = position
    return f'{i+1}{column_letter_mapping[j]}'
    # content = get_content_of_piece(curr_board, i, j)

In [ ]:
def match_template(template, img):
    _, max_similarity_score, _, _ = cv.minMaxLoc(cv.matchTemplate(img, template, cv.TM_CCOEFF_NORMED))
    return max_similarity_score

In [195]:
def extract_board(image):
    # Preprocess
    blue_channel = image[:, :, 0]
    median_blurred_img = cv.medianBlur(blue_channel, 3)
    gaussian_blurred_img = cv.GaussianBlur(median_blurred_img, (7, 7), 0) 
    # Segmentate
    edge_enhanced_img = cv.addWeighted(median_blurred_img, 1.5, gaussian_blurred_img, -0.5, 0)
    _, mask = cv.threshold(edge_enhanced_img, 80, 255, cv.THRESH_BINARY)
    # Refine
    kernel = np.ones((5, 5), np.uint8)
    mask_cleaned1 = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    mask_cleaned2 = cv.morphologyEx(mask_cleaned1, cv.MORPH_OPEN, kernel)
    mask_cleaned3 = cv.erode(mask_cleaned2, kernel, iterations=4)
    # Extract
    contours, _ = cv.findContours(mask_cleaned3, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE) 
    contours = sorted(contours, key=cv.contourArea, reverse=True)
    board_contour = contours[0]
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        aspect_ratio = w / h
        if 0.8 < aspect_ratio < 1.2: 
            board_contour = contour
            break
        
    x, y, w, h = cv.boundingRect(board_contour)
    cropped_board = image[y:y+h, x:x+w]
    resized_cropped_board = cv.resize(cropped_board, (1900, 1900))

    x_start, y_start = 250, 250  
    x_end, y_end = 1650, 1650  

    play_area = resized_cropped_board[y_start:y_end, x_start:x_end]
    resized_play_area = cv.resize(play_area, (1400, 1400))

    # Uncomment to see the intermediate results
    # img_with_all_contours = cv.drawContours(image.copy(), contours, -1, (255, 0, 0), 10)
    # img_with_board_contour = cv.drawContours(image.copy(), board_contour, -1, (255, 0, 0), 10) 
    # fig = plt.figure(figsize=(12, 12))
    # images = [
    #     (image, "Original Image"),
    #     (median_blurred_img, "Median Blur"),
    #     (gaussian_blurred_img, "Gaussian Blur"),
    #     (edge_enhanced_img, "Edge enchancement"),
    #     (mask, "Mask"),
    #     (mask_cleaned1, "Mask Cleaned Closing"),
    #     (mask_cleaned2, "Mask Cleaned Opening"),
    #     (mask_cleaned3, "Mask Cleaned Erosion"),
    #     (img_with_all_contours, "Img with all contours"),
    #     (img_with_board_contour, "Img with largest contour"),
    #     (resized_cropped_board, "Resized cropped board"),
    #     (resized_play_area, "Resized play area")
    # ]
    # plot_images(fig, images)
    return resized_play_area
   
    

In [308]:
moves_per_game = 50

def resolve_game(game):
    correct = 0
    prev_img, curr_img = None, None
    for move in range(1, moves_per_game+1):
        if move == 1:
            prev_img = cv.imread('imagini_auxiliare/01.jpg')
            curr_img = cv.imread(f'antrenare/{game}_0{move}.jpg')
            actual_position, actual_content = open(f'antrenare/{game}_0{move}.txt', 'r').read().split(" ")
        else:
            append_zero_to_prev = '0' if move <= 10 else ''
            append_zero_to_curr = '0' if move < 10 else ''
            prev_img = cv.imread(f'antrenare/{game}_{append_zero_to_prev}{move-1}.jpg')
            curr_img = cv.imread(f'antrenare/{game}_{append_zero_to_curr}{move}.jpg')
            actual_position, actual_content = open(f'antrenare/{game}_{append_zero_to_curr}{move}.txt', 'r').read().split(" ")

        prev_board = extract_board(prev_img)
        curr_board = extract_board(curr_img)
        position = get_position_of_newly_added_piece(prev_board, curr_board)
        content = get_content_of_piece(curr_board, position[0], position[1])

        result_position = position_and_content(position, content)
        if result_position != actual_position:
            print(f'Expected: {actual_position}, got: {result_position} for img {game}_{move}')
        else:
            correct += 1

    print(f'Correct: {correct}/{moves_per_game}')
    print(f'Accuracy: {correct/moves_per_game * 100}%') 
    
print('Task1')
for game in range(1,5):
    print(f'Game {game}')
    resolve_game(game)
    print('-------------------')
    

Task1
Game 1
Correct: 50/50
Accuracy: 100.0%
-------------------
Game 2
Correct: 50/50
Accuracy: 100.0%
-------------------
Game 3
Correct: 50/50
Accuracy: 100.0%
-------------------
Game 4
Correct: 50/50
Accuracy: 100.0%
-------------------
